<a href="https://colab.research.google.com/github/Xicotrainer/Python/blob/master/3_3_NLP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RECOMIENDAME UN LIBRO
(**Base de datos de libros en inglés de goodreads**)
___
___

## 1 Carga de librerias, accesos y bases
___

In [ ]:
import pandas as pd
import string as strg
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Acceso
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/Colab Notebooks/1_Datasets'
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/1_Datasets
1.1_input_dataset.csv   1.3_clean_dataset.csv    1.5_encoding_dataset.csv
1.2_sample_dataset.csv  1.4_cluster_dataset.csv  1.6_pca_cluster_dataset.csv


In [ ]:
df = pd.read_csv('1.4_cluster_dataset.csv', index_col=0)

In [ ]:
df.head(3)

,ISBN,Name,Authors,Publisher,Rating,pagesNumber,CountsOfReview,PublishYear,Cluster
0,0439554896,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,Scholastic,4.42,352,244,2003,2
1,0976540606,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,Nimble Books,3.79,152,1,2005,0
2,0767910435,Bryson's Dictionary of Troublesome Words: A Wr...,Bill Bryson,Broadway Books,3.88,256,131,2004,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13405 entries, 0 to 13404
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ISBN            13405 non-null  object 
 1   Name            13405 non-null  object 
 2   Authors         13405 non-null  object 
 3   Publisher       13405 non-null  object 
 4   Rating          13405 non-null  float64
 5   pagesNumber     13405 non-null  int64  
 6   CountsOfReview  13405 non-null  int64  
 7   PublishYear     13405 non-null  int64  
 8   Cluster         13405 non-null  int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 1.0+ MB


___
___

## 2 Reconstrucción de variables categoricas.
___

### 2.1 Pre-procesamiento.

In [ ]:
num_df = df[list(df.columns[df.dtypes == object])]
num_df.sample(3)

,ISBN,Name,Authors,Publisher
6147,0060934727,Mike Nelson's Death Rat!,Michael J. Nelson,Dey Street Books
8640,0813108489,"The Land Beyond the Mountains (Kentuckians, #3)",Janice Holt Giles,University Press of Kentucky
1758,1892391368,The Line Between,Peter S. Beagle,Tachyon Publications


In [ ]:
# Concatenar los tres campos de nombre (N), autor (A) y editorial (P).

df['NAP'] = df.Name.str.cat(df.Authors, sep=" ").str.cat(df.Publisher, sep=" ")
df['NAP']

0        Harry Potter and the Chamber of Secrets (Harry...
1        Unauthorized Harry Potter Book Seven News: "Ha...
2        Bryson's Dictionary of Troublesome Words: A Wr...
3        The Lord of the Rings: Weapons and Warfare Chr...
4        The Lord of the Rings: Complete Visual Compani...
                               ...                        
13400    Thomas Paine and the Promise of America (Oxfor...
13401    Common Sense and Other Writings Thomas Paine B...
13402    Faith of My Fathers: A Family Memoir John McCa...
13403    Science Fiction: The Best of 2001 Robert Silve...
13404    Nebula Awards Showcase 2001 Robert Silverberg ...
Name: NAP, Length: 13405, dtype: object

In [ ]:
df['NAP'] = df['NAP'].str.lower()
df['NAP']

0        harry potter and the chamber of secrets (harry...
1        unauthorized harry potter book seven news: "ha...
2        bryson's dictionary of troublesome words: a wr...
3        the lord of the rings: weapons and warfare chr...
4        the lord of the rings: complete visual compani...
                               ...                        
13400    thomas paine and the promise of america (oxfor...
13401    common sense and other writings thomas paine b...
13402    faith of my fathers: a family memoir john mcca...
13403    science fiction: the best of 2001 robert silve...
13404    nebula awards showcase 2001 robert silverberg ...
Name: NAP, Length: 13405, dtype: object

In [ ]:
df['NAP'] = df['NAP'].apply(
    lambda linea: linea.translate(str.maketrans('', '', strg.punctuation))
    )
df['NAP']

0        harry potter and the chamber of secrets harry ...
1        unauthorized harry potter book seven news half...
2        brysons dictionary of troublesome words a writ...
3        the lord of the rings weapons and warfare chri...
4        the lord of the rings complete visual companio...
                               ...                        
13400    thomas paine and the promise of america oxford...
13401    common sense and other writings thomas paine b...
13402    faith of my fathers a family memoir john mccai...
13403    science fiction the best of 2001 robert silver...
13404    nebula awards showcase 2001 robert silverberg ...
Name: NAP, Length: 13405, dtype: object

___
___

### 2.2 Vectorización

In [ ]:
df['NAP'] = df['NAP'].apply(lambda registro: registro.split())
df['NAP']

0        [harry, potter, and, the, chamber, of, secrets...
1        [unauthorized, harry, potter, book, seven, new...
2        [brysons, dictionary, of, troublesome, words, ...
3        [the, lord, of, the, rings, weapons, and, warf...
4        [the, lord, of, the, rings, complete, visual, ...
                               ...                        
13400    [thomas, paine, and, the, promise, of, america...
13401    [common, sense, and, other, writings, thomas, ...
13402    [faith, of, my, fathers, a, family, memoir, jo...
13403    [science, fiction, the, best, of, 2001, robert...
13404    [nebula, awards, showcase, 2001, robert, silve...
Name: NAP, Length: 13405, dtype: object

In [ ]:
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [ ]:
df['NAP'] = df['NAP'].apply(
    lambda registro: [palabra for palabra in registro if palabra not in set(stopwords.words('english'))]
    )

df['NAP']

0        [harry, potter, chamber, secrets, harry, potte...
1        [unauthorized, harry, potter, book, seven, new...
2        [brysons, dictionary, troublesome, words, writ...
3        [lord, rings, weapons, warfare, chris, smith, ...
4        [lord, rings, complete, visual, companion, jud...
                               ...                        
13400    [thomas, paine, promise, america, oxford, port...
13401    [common, sense, writings, thomas, paine, barne...
13402    [faith, fathers, family, memoir, john, mccain,...
13403    [science, fiction, best, 2001, robert, silverb...
13404    [nebula, awards, showcase, 2001, robert, silve...
Name: NAP, Length: 13405, dtype: object

In [ ]:
pal_der = PorterStemmer()

df['NAP'] = df['NAP'].apply(
    lambda registro: [pal_der.stem(palabra) for palabra in registro]
    )

df['NAP']

0        [harri, potter, chamber, secret, harri, potter...
1        [unauthor, harri, potter, book, seven, news, h...
2        [bryson, dictionari, troublesom, word, writer,...
3        [lord, ring, weapon, warfar, chri, smith, houg...
4        [lord, ring, complet, visual, companion, jude,...
                               ...                        
13400    [thoma, pain, promis, america, oxford, portrai...
13401    [common, sens, write, thoma, pain, barn, nobl,...
13402    [faith, father, famili, memoir, john, mccain, ...
13403    [scienc, fiction, best, 2001, robert, silverbe...
13404    [nebula, award, showcas, 2001, robert, silverb...
Name: NAP, Length: 13405, dtype: object

In [ ]:
pal_der = WordNetLemmatizer()

df['NAP'] = df['NAP'].apply(
    lambda registro: [pal_der.lemmatize(palabra) for palabra in registro]
    )

df['NAP']

0        [harri, potter, chamber, secret, harri, potter...
1        [unauthor, harri, potter, book, seven, news, h...
2        [bryson, dictionari, troublesom, word, writer,...
3        [lord, ring, weapon, warfar, chri, smith, houg...
4        [lord, ring, complet, visual, companion, jude,...
                               ...                        
13400    [thoma, pain, promis, america, oxford, portrai...
13401    [common, sen, write, thoma, pain, barn, nobl, ...
13402    [faith, father, famili, memoir, john, mccain, ...
13403    [scienc, fiction, best, 2001, robert, silverbe...
13404    [nebula, award, showcas, 2001, robert, silverb...
Name: NAP, Length: 13405, dtype: object

In [ ]:
df['NAP'] = df['NAP'].apply(
    lambda registro:" ".join(registro)
    )

df['NAP']

0        harri potter chamber secret harri potter 2 jk ...
1        unauthor harri potter book seven news halfbloo...
2        bryson dictionari troublesom word writer guid ...
3        lord ring weapon warfar chri smith houghton mi...
4        lord ring complet visual companion jude fisher...
                               ...                        
13400    thoma pain promis america oxford portrait harv...
13401        common sen write thoma pain barn nobl classic
13402    faith father famili memoir john mccain william...
13403    scienc fiction best 2001 robert silverberg fan...
13404    nebula award showcas 2001 robert silverberg ma...
Name: NAP, Length: 13405, dtype: object

In [ ]:
vec_df = CountVectorizer()

___
___

### 2.3 Semejanza

In [ ]:
vec_df = vec_df.fit_transform(df['NAP'])
vec_df.shape

(13405, 16539)

In [ ]:
sem_df = cosine_similarity(vec_df,vec_df)
sem_df.shape

(13405, 13405)

In [ ]:
def orden_semejanza(ISBN):
    ind_sem = pd.Series(df.index, index=df['ISBN']).drop_duplicates()
    ind_sem = ind_sem[ISBN]

    sem = list(enumerate(sem_df[ind_sem]))
    sem = sorted(sem, key=lambda orden: orden[1], reverse=True)
   
    mask = [i[0] for i in sem]
    return df.iloc[mask].drop(columns = ['NAP'])

___
___

### 2.4 Recomendación

In [ ]:
ISBN = '0439554896'

consulta = orden_semejanza(ISBN)
consulta.head(11)

,ISBN,Name,Authors,Publisher,Rating,pagesNumber,CountsOfReview,PublishYear,Cluster
0,0439554896,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,Scholastic,4.42,352,244,2003,2
9674,1417751398,Harry Potter and the Half-blood Prince (Harry ...,J.K. Rowling,Turtleback Books,4.57,652,9,2006,3
7109,0439451930,Harry Potter and the Chamber of Secrets: A Mag...,Dena Neusner,Scholastic,4.36,5,3,2002,0
311,0826452329,J.K. Rowling's Harry Potter Novels: A Reader's...,Philip Nel,Bloomsbury Academic,3.59,96,9,2001,0
2288,1414306342,Looking for God in Harry Potter,John Granger,SaltRiver,4.12,234,95,2006,1
310,043932162X,Harry Potter Schoolbooks Box Set: Two Classic ...,J.K. Rowling,Arthur A. Levine,4.40,240,140,2001,1
2910,0723258058,Beatrix Potter's Journal,Beatrix Potter,Warne,4.36,32,42,2006,0
4805,0812694554,Harry Potter and Philosophy: If Aristotle Ran ...,David Baggett,Open Court,4.48,243,78,2004,1
3173,0722189141,The Secrets of Harry Bright,Joseph Wambaugh,Sphere,3.76,307,26,1987,1
9945,032301741X,Virtual Clinical Excursions for Potter Fundame...,Patricia A. Potter,Mosby,3.90,495,0,2001,3


In [ ]:
consulta[consulta['Cluster'] == 2].head(11)

,ISBN,Name,Authors,Publisher,Rating,pagesNumber,CountsOfReview,PublishYear,Cluster
0,0439554896,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,Scholastic,4.42,352,244,2003,2
2719,1585676160,Piccadilly Jim,P.G. Wodehouse,Harry N. Abrams,4.02,302,187,2004,2
7493,1585670219,Masters of Atlantis,Charles Portis,Harry N. Abrams,3.74,248,212,2000,2
8487,1585677647,Behold the Man,Michael Moorcock,Harry N. Abrams,3.78,144,220,2007,2
618,0609602195,The Barefoot Contessa Cookbook,Ina Garten,Clarkson Potter,4.16,256,178,1999,2
2444,1585672319,"The Mating Season (Jeeves, #9)",P.G. Wodehouse,Harry N. Abrams,4.26,272,290,2002,2
3740,0810987902,The Boy Who Couldn't Die,William Sleator,Harry N. Abrams,3.68,161,248,2005,2
4920,0425196992,"Shakespeare's Trollop (Lily Bard, #4)",Charlaine Harris,Berkley,3.87,194,368,2004,2
6816,0743225082,"Man and Boy (Harry Silver, #1)",Tony Parsons,Touchstone,3.57,368,443,2002,2
13347,0425201147,"Shakespeare's Counselor (Lily Bard, #5)",Charlaine Harris,Berkley,3.90,243,389,2005,2


> Ejemplo aleatorio

In [ ]:
df.sample(5)

,ISBN,Name,Authors,Publisher,Rating,pagesNumber,CountsOfReview,PublishYear,Cluster,NAP
5187,0399153691,"Treasure of Khan (Dirk Pitt, #19)",Clive Cussler,G.P. Putnam's Sons,3.91,552,336,2006,2,treasur khan dirk pitt 19 clive cussler gp put...
11288,0812589815,When The King Comes Home (A College of Magics),Caroline Stevermer,Tor Fantasy,3.51,256,46,2001,1,king come home colleg magic carolin steverm to...
6951,1569242739,Vegan Cupcakes Take Over the World: 75 Dairy-F...,Isa Chandra Moskowitz,Da Capo Lifelong Books,3.94,176,339,2006,2,vegan cupcak take world 75 dairyfre recip cupc...
11172,0618694064,Disappearances,Howard Frank Mosher,Mariner Books,3.93,272,41,2006,1,disappear howard frank mosher marin book
11310,0520085558,The Nietzsche Legacy in Germany: 1890 - 1990,Steven E. Aschheim,University of California Press,3.61,337,3,1994,1,nietzsch legaci germani 1890 1990 steven e asc...


In [ ]:
ISBN = '0060887966'

consulta = orden_semejanza(ISBN)
consulta.head(11)

,ISBN,Name,Authors,Publisher,Rating,pagesNumber,CountsOfReview,PublishYear,Cluster
155,0060887966,The Alchemist,Paulo Coelho,HarperOne,3.87,192,76,2006,0
217,0062513346,The Valkyries,Paulo Coelho,HarperOne,3.31,212,87,1996,0
218,0060930136,The Fifth Mountain,Paulo Coelho,HarperOne,3.62,256,75,2000,1
7670,0060528001,The Devil and Miss Prym,Paulo Coelho,HarperOne,3.61,205,80,2007,0
9698,0062501755,The Diary of a Magus: The Road to Santiago,Paulo Coelho,HarperOne,3.65,226,5,1992,0
216,006251279X,The Pilgrimage: A Contemporary Quest for Ancie...,Paulo Coelho,HarperOne,3.65,272,95,1995,1
160,006019250X,The Illustrated Alchemist: A Fable about Follo...,Paulo Coelho,HarperCollins Publishers,3.87,198,33,1998,0
7669,006133880X,The Witch of Portobello,Paulo Coelho,HarperCollins,3.54,268,121,2007,1
7671,0007114370,Paulo Coelho: Confessions of a Pilgrim,Juan Arias,HarperCollins Publishers,3.92,254,30,2001,1
3973,1421502224,Fullmetal Alchemist: The Abducted Alchemist (F...,Makoto Inoue,VIZ Media LLC,4.57,240,19,2006,1


$Z_{cal} = \frac{(\bar{x_A}-\bar{x_B})-(\bar{\mu_A}-\bar{\mu_B})}{\sqrt{\frac{s^2_A}{n_A} +\frac{s^2_B}{n_B}  }}$

In [ ]:
consulta[consulta['Cluster'] == 0].head(11)

,ISBN,Name,Authors,Publisher,Rating,pagesNumber,CountsOfReview,PublishYear,Cluster
155,0060887966,The Alchemist,Paulo Coelho,HarperOne,3.87,192,76,2006,0
217,0062513346,The Valkyries,Paulo Coelho,HarperOne,3.31,212,87,1996,0
7670,0060528001,The Devil and Miss Prym,Paulo Coelho,HarperOne,3.61,205,80,2007,0
9698,0062501755,The Diary of a Magus: The Road to Santiago,Paulo Coelho,HarperOne,3.65,226,5,1992,0
160,006019250X,The Illustrated Alchemist: A Fable about Follo...,Paulo Coelho,HarperCollins Publishers,3.87,198,33,1998,0
6631,0061351342,El Alquimista: Edicion Illustrada: Edicion Ill...,Paulo Coelho,Rayo,3.87,192,5,2007,0
156,142150460X,"Fullmetal Alchemist, Vol. 9 (Fullmetal Alchemi...",Hiromu Arakawa,VIZ Media LLC,4.58,192,156,2006,0
158,1421504596,"Fullmetal Alchemist, Vol. 8 (Fullmetal Alchemi...",Hiromu Arakawa,VIZ Media LLC,4.57,192,159,2006,0
3693,0062511114,A Manual for Living,Epictetus,HarperOne,4.23,96,15,1994,0
4632,0060761539,What Christians Believe,C.S. Lewis,HarperOne,4.24,98,72,2005,0


In [ ]:
df.to_csv('1.5_encoding_dataset.csv')